# TUIA / AAII - TP1: Redes Densas y Convoluciones - Problema 1

## *::::: Modelo de Regresión para predecir el Índice de Rendimiento Académico :::::*

# Procedimientos Previos

## Preparación del entorno

### Librerías

In [46]:
# Datasets
import gdown
import zipfile

# Data
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


# Visualizaciones
import matplotlib.pyplot as plt

# Redes Neuronales
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.layers import Input, Dense

In [1]:
# Importar desde Google Drive
!pip install -q gdown

### GPU

In [ ]:
# Configurar para que TensorFlow utilice la GPU por defecto
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar para que TensorFlow asigne memoria dinámicamente
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # Especificar la GPU por defecto
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Manejar error
        print(e)

## Dataset

El dataset proporcionado incluye las siguientes variables para cada estudiante:

* **Hours Studied**: El número total de horas dedicadas al estudio por cada estudiante.
* **Previous Scores**: Las puntuaciones obtenidas por los estudiantes en exámenes previos.
* **Extracurricular Activities**: Si el estudiante participa en actividades extracurriculares (Sí o No).
* **Sleep Hours**: El número promedio de horas de sueño que el estudiante tuvo por día.
* **Sample Question Papers Practiced**: El número de cuestionarios de muestra que el estudiante practicó.

Además, el dataset incluye la variable objetivo:
* **Performance Index**: Un índice que representa el rendimiento académico general de cada estudiante, redondeado al entero más cercano. Este índice varía de 10 a 100, donde valores más altos indican un mejor rendimiento.

### Descarga

In [6]:
# TP1-1-student_performance.zip
gdown.download('https://drive.google.com/uc?id=1mfpXVLqDJah-sO0CF29LjKUz5NtKjZqc')

Downloading...
From: https://drive.google.com/uc?id=1mfpXVLqDJah-sO0CF29LjKUz5NtKjZqc
To: /content/TP1-1-student_performance.zip
100%|██████████| 49.7k/49.7k [00:00<00:00, 26.0MB/s]


'TP1-1-student_performance.zip'

### Descompresión

In [9]:
with zipfile.ZipFile('TP1-1-student_performance.zip', 'r') as zip_ref:
    zip_ref.extractall('./TP1-1')

### Carga

In [92]:
# Cargamos el dataset
df = pd.read_csv("TP1-1/Student_Performance.csv")
df

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,Yes,9,1,91.0
1,4,82,No,4,2,65.0
2,8,51,Yes,7,2,45.0
3,5,52,Yes,5,2,36.0
4,7,75,No,8,5,66.0
...,...,...,...,...,...,...
9995,1,49,Yes,4,2,23.0
9996,7,64,Yes,8,5,58.0
9997,6,83,Yes,8,5,74.0
9998,9,97,Yes,7,0,95.0


## Análisis Exploratorio

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
cols = df.columns

for col in cols:
    print("--------------------")
    print("Columna: ", col)
    print("Cantidad de valores nulos: ", df[col].isnull().sum())
    print("Cantidad de valores únicos: ", df[col].nunique())
    print("Tipo de dato: ", df[col].dtype)
    # Si no es numérica, continuar con la siguiente columna
    if df[col].dtype == "object":
        print("--------------------\n")
        continue
    # Si es una columna categórica, mostrar la cantidad de veces que aparece cada valor
    if df[col].nunique() < 10:
        print("Valores únicos: ", df[col].unique())
        print("Cantidad de veces que aparece cada valor: ", df[col].value_counts())
    else:
        # Si es una columna numérica, mostramos media, desvío estándar, mínimo, máximo...
        print("Media: ", df[col].mean())
        print("Desvío estándar: ", df[col].std())
        print("Mínimo: ", df[col].min())
        print("Máximo: ", df[col].max())
    print("--------------------\n")

## Conjuntos de Train y Test

In [105]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [106]:
df_train.shape, df_test.shape

((8000, 6), (2000, 6))

## Procesamiento en Train

### Renombrar Columnas

In [107]:
def renombrar_columnas(df):
    """
    Renombra las columnas de un DataFrame con nombres abreviados.

    Parámetros:
    - df: DataFrame: El DataFrame que contiene los datos.

    Retorna:
    - df: DataFrame: El DataFrame con las columnas renombradas.
    """
    # Diccionario con los nombres y sus reemplazos
    nuevos_nombres = {
    'Hours Studied': 'Hs_Studied',
    'Previous Scores': 'Prev_Score',
    'Extracurricular Activities': 'Extra_Act',
    'Sleep Hours': 'Sleep_Hs',
    'Sample Question Papers Practiced': 'Practiced',
    'Performance Index': 'Index'
    }
    df.rename(columns=nuevos_nombres, inplace=True)
    return df

In [108]:
df_train = renombrar_columnas(df_train)

### Filas duplicadas

In [98]:
duplicadas = df_train.duplicated().any()
print("Hay filas duplicadas.") if duplicadas.any() else print("No hay filas duplicadas.")

Hay filas duplicadas.


In [99]:
filas_duplicadas = df_train[df_train.duplicated()]
filas_duplicadas

,Hs_Studied,Prev_Score,Extra_act,Sleep_Hs,Practiced,Index
4053,9,58,Yes,5,2,56.0
3405,4,43,No,6,7,24.0
622,2,62,Yes,9,4,40.0
5020,6,88,No,4,6,75.0
2647,1,57,No,8,0,31.0
...,...,...,...,...,...,...
7158,6,46,No,5,3,34.0
2511,9,50,No,7,8,45.0
202,9,52,No,5,9,48.0
9954,6,97,No,8,7,92.0


In [109]:
def eliminar_registros_duplicados(df):
    """
    Elimina los registros duplicados de un DataFrame.

    Parámetros:
    - df: DataFrame: El DataFrame que se va a limpiar de registros duplicados.

    Retorna:
    - df_clean: DataFrame: El DataFrame sin registros duplicados.
    """
    df_clean = df.drop_duplicates()
    return df_clean

In [110]:
df_train = eliminar_registros_duplicados(df_train)

In [111]:
df_train.shape

(7924, 6)

### Recategorización de Variables

In [112]:
def recategorizar_variables(df):
    """
    Recategoriza las variable deseadas en un DataFrame.

    Parámetros:
    - df: DataFrame: El DataFrame que contiene los datos.

    Retorna:
    - df_recategorizado: DataFrame: El DataFrame con las variables recategorizadas.
    """
    df_recategorizado = df.copy()

    # Convierte "Yes" en 1 y "No" en 0.
    df_recategorizado['Extra_Act'] = df_recategorizado['Extra_Act'].map({'Yes': 1, 'No': 0})

    return df_recategorizado

In [113]:
df_train = recategorizar_variables(df_train)

### Conjuntos X e y

In [57]:
def separar_variables(df, target_column):
    """
    Separa las variables explicativas y la variable objetivo de un DataFrame.

    Parámetros:
    - df: DataFrame: El DataFrame que contiene los datos.
    - target_column: str: El nombre de la columna que se utilizará como variable objetivo.

    Retorna:
    - X: DataFrame: El DataFrame que contiene las variables explicativas.
    - y: DataFrame: El DataFrame que contiene la variable objetivo.
    """
    # Variables explicativas (todas las columnas excepto la columna objetivo)
    X = df.drop([target_column], axis=1).copy()

    # Variable objetivo (solo la columna objetivo)
    y = df[[target_column]].copy()

    return X, y

In [72]:
X_train, y_train = separar_variables(df_train, 'Index')

In [74]:
X_train.shape, y_train.shape

((8000, 5), (8000, 1))

## Visualizaciones

# Modelo

## Procesamiento en Test